# BES Analysis in the Radial Direction

Previous work has been in the vertical direction.

Import neccesary packages

In [1]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
from plotting_functions_BES import *
import seaborn as sns
import pandas as pd

Read BES data

In [2]:
# for BES data
# (R, z) locations for the BES channels (view location)
apdpos = np.asarray(xr.open_dataarray('Data\\shot29378_apdpos.nc'))

fluct_data_from_file = xr.open_dataarray('Data\\shot29378_LH_fluct_data.nc')

bes_time = np.asarray(fluct_data_from_file.coords['time']) # bes_time[0:992499]
fluct_data = np.asarray(fluct_data_from_file) # fluct_data[0:31][0:992499]

Define interesting timeslices

In [3]:
regions = [[0.16,0.24], [0.36,0.54], [0.54,0.68]]

Function to compute 2D FFT of BES data using inner-most four channels of specified row.

In [4]:
def get_kf_spec_radial(row, timeslice): # Row is int from 0 to 3 . timeslice = [t1, t2], a 2x1 array
    channels = [ch for ch in range(row*8, row*8+4)] # Get first four channels of the row
    region = regions.index(timeslice)+1 # Get region number as referenced in project so far
    
    space_array = [apdpos[i][0] for i in channels] # Get R-coordinates of each channel
    space_array = np.asarray(space_array)
    spec = []
    for ch in channels: # FFT each channel in time
        fft = get_channel_fft(29378, bes_time, fluct_data, ch, timeslice, "channel_fft")
        f_transform = fft[1]
        spec.append(f_transform) # Each row of spec corresponds to a channel.
    f_arr = fft[0] # Frequency array is the same for all of them so just save any one.
    
    spec = np.asarray(spec)
    spec = np.transpose(spec) # Now each row is a time point as required by calc_kspecs
    calc = calc_kspec(spec, space_array) # Get k-f spectrum
    
    kf_matrix = calc[0] # This contains the transform data
    k_arr = calc[1] # This is the array of wavenumbers
    
    return f_arr, k_arr, kf_matrix

Function for plotting the k-f spectra as a heatmap. This is same as vertical version just with labels changed from column to row.

In [5]:
def plot_kf_spec_radial(region, row, f_arr, k_arr, kf_matrix, fmin=0.0, fmax=None, save=True):
    if fmax == None:
        fmax = np.max(f_arr)
    # Only need to plot a section of the spectrum. At least half not needed.
    kf_matrix = kf_matrix[(np.abs(f_arr - fmin)).argmin():(np.abs(f_arr - fmax)).argmin()]
    f_arr = f_arr[(np.abs(f_arr - fmin)).argmin():(np.abs(f_arr - fmax)).argmin()]
    
    # Convert to Pandas DataFrame with rounded frequencies as indices for easier of plotting.
    kf_matrix_plot = pd.DataFrame(kf_matrix, index=np.around(f_arr,0), columns=np.around(k_arr,1))

    # For only plotting tick at every 50 kHz
    interval = int(np.abs(f_arr - 50.0).argmin())
    
    # Plot log of values so small enough range for features to be visible
    sns.heatmap(np.log(np.abs(kf_matrix_plot)**2)[::-1], yticklabels=interval, cmap="plasma", cbar_kws={"label": r"$\log\vert S(f,k)\vert^2$"})
    plt.title("Region " + str(region) + ", t=" + str(regions[region-1]) + " s, Row " + str(row) +  ", Left 4 Channels" + " (radial)")
    plt.ylabel("Frequency [kHz]")
    plt.xlabel(r"Wavenumber [m$^{-1}$]")
    if save == True:
        plt.savefig("Plots/Good kf_specs/Radial/" + datestamp + "/kf_spec_reg_" + str(region) + "_row_" + str(row) + "_l4.png", format="png", bbox_inches="tight", dpi=300)
    else:
        plt.show()
    plt.close()

In [6]:
"""for region in range(3):
    for row in range(4):
        tmp = get_kf_spec_radial(row,regions[region])
        plot_kf_spec_radial(region+1, row+1, tmp[0], tmp[1], tmp[2], fmax=250.0, save=False)"""

'for region in range(3):\n    for row in range(4):\n        tmp = get_kf_spec_radial(row,regions[region])\n        plot_kf_spec_radial(region+1, row+1, tmp[0], tmp[1], tmp[2], fmax=250.0, save=False)'

Function for summing contributions for each wavenumber OR frequency to produce a spectrum of FFT against frequency OR wavenumber

In [7]:
def kf_sum(kf_matrix, keep_var): # keep_var is the variable not summed over. E.g. if f it sums ks for each f.
    if keep_var == "k":
        spec = np.sum(kf_matrix, axis=0)
    elif keep_var == "f":
        spec = np.sum(kf_matrix, axis=1)
    else:
        raise ValueError("Bad argument keep_var = " + keep_var)
    return spec

Function to sum points over a 1D array in a given distinct window. new_freqs is 1/window_size of freqs

In [8]:
def spec_box_sum(spec, freqs, window_length):
    spec_smooth = []
    new_freqs = []
    window_hw = int(np.floor(window_length/2))

    for i in range(int(np.ceil(window_length/2)),len(spec)-window_hw,window_length): # From first to last midpoint
        window_sum = 0
        for j in range(i-window_hw, i+window_hw+1): # +1 to include the end point
            window_sum += spec[j]
        spec_smooth.append(window_sum)
        new_freqs.append(freqs[i])
    return new_freqs, spec_smooth

Function to do same as spec_box_sum but for 2D array

In [9]:
def kf_spec_box_sum(f_arr, k_arr, kf_matrix, window_length):
    kf_matrix = np.transpose(kf_matrix)
    new_matrix = []
    
    for i in kf_matrix:
        summed = spec_box_sum(i, f_arr, window_length)
        new_matrix.append(summed[1])
    return np.asarray(summed[0]), k_arr, np.transpose(np.asarray(new_matrix))

Make kf spectra plots where frequencies summed over a number of points.

In [10]:
"""for region in range(3):
    for row in range(4):
        tmp = get_kf_spec_radial(row,regions[region])
        blocked = kf_spec_box_sum(tmp[0], tmp[1], tmp[2], 10)
        plot_kf_spec_radial(region+1, row+1, blocked[0], blocked[1], blocked[2], fmax=250, save=False)"""

'for region in range(3):\n    for row in range(4):\n        tmp = get_kf_spec_radial(row,regions[region])\n        blocked = kf_spec_box_sum(tmp[0], tmp[1], tmp[2], 10)\n        plot_kf_spec_radial(region+1, row+1, blocked[0], blocked[1], blocked[2], fmax=250, save=False)'

Function to convert $S(f,k) \to S(k|f)$

In [11]:
def normalise_kf(f_arr, kf_matrix, fmin=0.0, fmax=None):
    if fmax == None:
        fmax = np.max(f_arr)

    kf_matrix = kf_matrix[(np.abs(f_arr - fmin)).argmin():(np.abs(f_arr - fmax)).argmin()]
    f_arr = f_arr[(np.abs(f_arr - fmin)).argmin():(np.abs(f_arr - fmax)).argmin()]
    spec = np.transpose(kf_matrix)/np.sum(kf_matrix,axis=1)

    return f_arr, np.transpose(spec)

In [12]:
def plot_kf_spec_rad_cond(region, col, f_arr, k_arr, kf_matrix, fmin=0.0, fmax=None, save=True):
    if fmax == None:
        fmax = np.max(f_arr)
    # Only need to plot a section of the spectrum. At least half not needed.
    kf_matrix = kf_matrix[(np.abs(f_arr - fmin)).argmin():(np.abs(f_arr - fmax)).argmin()]
    f_arr = f_arr[(np.abs(f_arr - fmin)).argmin():(np.abs(f_arr - fmax)).argmin()]
    
    # Convert to array with rounded frequencies for easier of plotting.
    kf_matrix_plot = pd.DataFrame(kf_matrix, index=np.around(f_arr,0), columns=np.around(k_arr,1))
    
    # For only plotting tick at every 50 kHz
    interval = int(np.abs(f_arr - 50.0).argmin())

    # Plot log of values so small enough range for features to be visible
    sns.heatmap(np.log(np.abs(kf_matrix_plot)**2)[::-1], yticklabels=interval, cmap="plasma", cbar_kws={"label": r"$\log\vert S(k|f)\vert^2$"})
    plt.title("Region " + str(region) + ", t=" + str(regions[region-1]) + " s, Column " + str(col) + " (radial)")
    plt.ylabel("Frequency [kHz]")
    plt.xlabel(r"Wavenumber [m$^{-1}$]")
    if save == True:
        plt.savefig("Plots/kf-conditional/Radial/" +  datestamp() + "/kf_cond_rad_spec_reg_" + str(region) + "_col_" + str(col) + ".png", format="png", bbox_inches="tight", dpi=300)
    else:
        plt.show()
    plt.close()

Produce plots of $S(k|f)$

In [14]:
for region in range(3):
    for row in range(4):
        tmp = get_kf_spec_radial(row,regions[region])
        spec = normalise_kf(tmp[0], tmp[2], fmax=250.0)
        blocked = kf_spec_box_sum(spec[0], tmp[1], spec[1], 10)
        plot_kf_spec_rad_cond(region+1, row+1, blocked[0], blocked[1], blocked[2], fmax=250, save=True)